## Lassoの理論
Lassoの損失関数
$$L=argmin_w(\frac{1}{2n} ||y-Xw||^2_2 + \alpha||w||_1)$$

Lassoの更新式
$$\frac{\partial L}{\partial w} = \frac{1}{n}X^T(Xw-y)+\alpha$$
(但し、$\alpha$の符号はwに依存するため上記は適切な式を表現できていない。この処理は後程行うため、便宜上$\alpha$としている。)

上記の更新式をより具体的に書くと
$$\frac{\partial L}{\partial w} = 
\begin{pmatrix}
1 & \dots & 1 \\
x_{11} & \dots & x_{n1} \\
\vdots & \ddots & \vdots \\
x_{1d} & \dots & x_{nd} \\
\end{pmatrix}
\begin{pmatrix}
\sum_{j=0}^{d} w_j x_{1j} - y_1+\alpha\\
\sum_{j=0}^{d} w_j x_{2j} - y_2+\alpha\\
\vdots\\
\sum_{j=0}^{d} w_j x_{nj} - y_n+\alpha\\
\end{pmatrix}$$
$$ ∴\frac{\partial L}{\partial w_k} = \frac{1}{n}\sum_{i=1}^{n}(x_{ik}\sum_{j=1}^{d} w_j x_{ij} - y_i) +\alpha \hspace{20mm} (0≤k≤m)$$
Lの最小値が知りたいため、
$$ \frac{\partial L}{\partial w_k} = 0 $$
を$w_k$について解くと、
$$ \sum_{i=1}^{n} x_{ik} \sum_{j≠k}^{d} (w_j x_{ij} - y_i) + w_k x_{ik} + n\alpha = 0 $$
$$ \sum_{i=1}^{n}x_{ik}\sum_{j≠k}^{d} (w_j x_{ij} - y_i) + w_k \sum_{i=1}^{n}x_{ik}^2 + n\alpha = 0 $$
$$ ∴w_k = \frac{\sum_{i=1}^{n}x_{ik}\sum_{j≠k}^{d} (y_i - w_j x_{ij})-n\alpha}{\sum_{i=1}^{n}x_{ik}^2} $$

続いて、$\alpha$の符号についての処理を行う。そのため、右偏微分並びに左偏微分の場合わけを(軟判定しきい値関数で)行う。

右偏微分の式は以下の通りになる。
$$w_k^+ = \frac{\sum_{i=1}^{n}x_{ik}\sum_{j≠k}^{d} (y_i - w_j x_{ij})-n\alpha}{\sum_{i=1}^{n}x_{ik}^2}$$

左偏微分の式は以下の通りになる。
$$w_k^- = \frac{\sum_{i=1}^{n}x_{ik}\sum_{j≠k}^{d} (y_i - w_j x_{ij})+n\alpha}{\sum_{i=1}^{n}x_{ik}^2}$$

右偏微分の式はw>0の時に成立する式であるため、以下のような条件が成立する。
$$\frac{\sum_{i=1}^{n}x_{ik}\sum_{j≠k}^{d} (y_i - w_j x_{ij})-n\alpha}{\sum_{i=1}^{n}x_{ik}^2} > 0$$

同様のことが左偏微分の式にも言えるため、
$$\frac{\sum_{i=1}^{n}x_{ik}\sum_{j≠k}^{d} (y_i - w_j x_{ij})+n\alpha}{\sum_{i=1}^{n}x_{ik}^2} < 0$$
といった式が成立する。

よって、上記の二つの範囲に属さない範囲においては更新されない。
$$-n\alpha < \sum_{i=1}^{n}x_{ik}\sum_{j≠k}^{d} (y_i - w_j x_{ij}) < n\alpha$$

これがLassoにおいて重要性の低いパラメータを0とすることができる理由である。

## Lassoの実装

In [1]:
# pip install mglearn
import mglearn

In [34]:
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.linear_model import Lasso
from sklearn.linear_model import LassoCV

In [3]:
# テストデータを用いる
# X: 104種類の特徴量が各506個ずつ
X, y = mglearn.datasets.load_extended_boston()

In [46]:
class MyLasso(LassoCV):
  def __init__(self, alpha=1.0, max_iter=1000, positive = False):
    self.alpha_ = alpha
    self.max_iter = max_iter
    self.coef_ = None
    self.intercept_ = None
    self.positive = positive

  # 軟判定しきい値関数(を利用しやすく改善したもの)
  def SoftThr(self, top, bottom):
    if top > self.n * self.alpha_:
      return (top - self.n * self.alpha_) / bottom
    elif top < -self.n * self.alpha_ and not self.positive:
      return (top + self.n * self.alpha_) / bottom
    else:
      return 0

  # wの更新
  def update(self, X, y):
    # バイアスの更新
    self.intercept_ = sum((y - X@self.coef_)) / self.n
    for k in range(self.d):
      coef = self.coef_
      # Σ(j!=k)の実現 (上記説明文参照)
      coef[k] = 0
      top = (y - self.intercept_ - X@coef)@X[:, k]
      bottom = sum(X[:, k] ** 2)
      # 更新すべき範囲の値かどうかの判別
      self.coef_[k] = self.SoftThr(top, bottom)
    
  def fit(self, X, y):
    self.n, self.d = X.shape
    self.coef_ = np.zeros(self.d)
    for _ in tqdm(range(self.max_iter)):
      self.update(X, y)
      
  def predict(self, X):
    return X@self.coef_ + self.intercept_
  
  # 決定係数を元にして計算を行う。
  def score(self, X, y):
    return 1 - sum((y - self.predict(X)) ** 2) / sum((y - np.mean(y)) ** 2)


intercept_やscore(X, y)は僅かな誤差を含むものの、ほとんど同じ値が出力される

In [47]:
lasso = MyLasso(positive=True)
lasso.fit(X, y)
lasso.intercept_ # 25.29637815194983
lasso.score(X, y) # 0.24391530654326432

100%|██████████| 1000/1000 [00:05<00:00, 183.15it/s]


0.1455708274261901

In [48]:
lasso2 = Lasso(alpha=1.0, positive=True)
lasso2.fit(X, y)
lasso2.intercept_ # 25.296295778633414
lasso2.score(X, y) # 0.24391102209551152

0.14557082742618854